In [2]:
# add local directory to import path
import os
import sys
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [17]:
#### --- Standard imports ------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import metrics
import xgboost

from IPython.display import HTML

from importlib import reload

# local import
import nflplot
import nflutil
import nfl_bdb22

# ======== NOTEBOOK SETTINGS ====================
flag_run_modelsearch = False  # True = run CV search, False = use fixed parameters

In [4]:
game_df = pd.read_csv('csv/games.csv')
play_df = pd.read_csv('csv/plays.csv')
pff_df = pd.read_csv('csv/PFFScoutingData.csv')

In [5]:
year = 2018
track_df = nflutil.transform_tracking_data(pd.read_csv(f'csv/tracking{year}.csv'))

# Get plays with a fair catch

In [16]:
track_df.loc[(track_df.gameId==2018090600) & (track_df.playId==973), 'event'].unique()

array(['None', 'ball_snap', 'punt', 'fair_catch'], dtype=object)

In [7]:
pff_df.loc[pff_df.kickContactType.isin(['BC','CC','MBDR']), ['gameId','playId', 'kickContactType']]

,gameId,playId,kickContactType
1,2018090600,366,CC
5,2018090600,973,CC
11,2018090600,1989,CC
13,2018090600,2406,CC
14,2018090600,2599,CC
...,...,...,...
19957,2021010315,1359,CC
19958,2021010315,1479,CC
19964,2021010315,2235,CC
19967,2021010315,2497,CC


In [8]:
(pff_df.loc[pff_df.kickContactType.isin(['BC','CC','MBDR']), ['gameId','playId','kickContactType']]
 .merge(play_df.loc[play_df.specialTeamsResult=='Fair Catch', ['gameId','playId']], how='inner', on=['gameId','playId'])
 ['kickContactType']
 .value_counts())

CC      1618
BC        10
MBDR       1
Name: kickContactType, dtype: int64

In [9]:
(pff_df.loc[pff_df.kickContactType.isin(['CC', 'BC', 'MBDR']), ['gameId','playId','kickContactType']]
 .merge(play_df.loc[play_df.specialTeamsResult=='Fair Catch', ['gameId','playId']], how='inner', on=['gameId','playId'])
 .merge(track_df.groupby(['gameId','playId','frameId']).head(1), how='inner', on=['gameId','playId'])
 [['gameId','playId','frameId','event']]
 .query('event != "None"')
 .groupby('event')
 .count()
)

,gameId,playId,frameId
event,,,
ball_snap,574,574,574
fair_catch,577,577,577
line_set,51,51,51
man_in_motion,3,3,3
out_of_bounds,2,2,2
penalty_flag,1,1,1
play_action,1,1,1
punt,577,577,577
punt_play,6,6,6


In [10]:
(track_df.merge(track_df.loc[track_df.event=="fair_catch", ['gameId','playId']], how='inner',on=['gameId','playId'])
 .groupby(['gameId','playId','frameId']).head(1)
 .query('event != "None"')['event']
 .value_counts())

fair_catch                  589
punt                        586
ball_snap                   583
line_set                     53
punt_received                 9
punt_play                     6
punt_land                     5
man_in_motion                 3
kickoff                       2
punt_downed                   2
out_of_bounds                 2
free_kick                     1
kickoff_land                  1
fumble_offense_recovered      1
penalty_flag                  1
play_action                   1
Name: event, dtype: int64

In [13]:
play_df.loc[play_df.specialTeamsResult=='Fair Catch', ['gameId','playId']]

,gameId,playId
5,2018090600,973
13,2018090600,2406
56,2018090900,4325
75,2018090901,2907
76,2018090901,3215
...,...,...
19933,2021010314,1771
19949,2021010315,283
19950,2021010315,386
19951,2021010315,553


In [31]:
reload(nfl_bdb22)

<module 'nfl_bdb22' from 'c:\\Users\\pschl\\Documents\\data_science\\NFL\\kaggle_nfl_bdb_2022\\nfl_bdb22.py'>

In [25]:
nfl_bdb22.prep_get_modeling_frames(track_df, 
                                   play_df=play_df, 
                                   pff_df=pff_df, 
                                   play_end_event_name='fair_catch'
                                   )

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,team,frameId,gameId,playId,playDirection
0,2018-12-30T21:58:01.300,27.30,23.50,2.02,1.10,0.21,95.17,106.31,punt,34723.0,Britton Colquitt,4.0,P,away,31,2018123000,892,left
1,2018-12-30T21:58:01.400,27.49,23.44,1.87,1.35,0.19,94.12,108.53,None,34723.0,Britton Colquitt,4.0,P,away,32,2018123000,892,left
2,2018-12-30T21:58:01.500,27.64,23.38,1.65,1.71,0.17,92.34,112.06,None,34723.0,Britton Colquitt,4.0,P,away,33,2018123000,892,left
3,2018-12-30T21:58:01.600,27.78,23.31,1.46,2.01,0.15,86.62,119.82,None,34723.0,Britton Colquitt,4.0,P,away,34,2018123000,892,left
4,2018-12-30T21:58:01.700,27.89,23.23,1.34,2.24,0.14,84.97,130.30,None,34723.0,Britton Colquitt,4.0,P,away,35,2018123000,892,left
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609725,2018-09-11T04:46:10.200,84.58,39.75,7.68,3.67,0.90,NaN,NaN,None,NaN,football,NaN,NaN,football,76,2018091001,3121,left
609726,2018-09-11T04:46:10.300,85.43,39.70,7.36,3.60,0.85,NaN,NaN,None,NaN,football,NaN,NaN,football,77,2018091001,3121,left
609727,2018-09-11T04:46:10.400,86.16,39.68,5.48,5.61,0.73,NaN,NaN,None,NaN,football,NaN,NaN,football,78,2018091001,3121,left
609728,2018-09-11T04:46:10.500,86.86,39.63,4.95,5.37,0.70,NaN,NaN,None,NaN,football,NaN,NaN,football,79,2018091001,3121,left


In [37]:
nfl_bdb22.prep_get_modeling_frames(track_df, 
                                   play_df=play_df, 
                                   pff_df=pff_df, 
                                   play_end_event_name='fair_catch'
                                   ).pipe(nfl_bdb22.feat_timeToCatch, catch_type='fair_catch')

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,team,frameId,gameId,playId,playDirection,timeToCatch
0,2018-12-30T21:58:01.300,27.30,23.50,2.02,1.10,0.21,95.17,106.31,punt,34723.0,Britton Colquitt,4.0,P,away,31,2018123000,892,left,4.5
1,2018-12-30T21:58:01.400,27.49,23.44,1.87,1.35,0.19,94.12,108.53,None,34723.0,Britton Colquitt,4.0,P,away,32,2018123000,892,left,4.4
2,2018-12-30T21:58:01.500,27.64,23.38,1.65,1.71,0.17,92.34,112.06,None,34723.0,Britton Colquitt,4.0,P,away,33,2018123000,892,left,4.3
3,2018-12-30T21:58:01.600,27.78,23.31,1.46,2.01,0.15,86.62,119.82,None,34723.0,Britton Colquitt,4.0,P,away,34,2018123000,892,left,4.2
4,2018-12-30T21:58:01.700,27.89,23.23,1.34,2.24,0.14,84.97,130.30,None,34723.0,Britton Colquitt,4.0,P,away,35,2018123000,892,left,4.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609725,2018-09-11T04:46:10.200,84.58,39.75,7.68,3.67,0.90,NaN,NaN,None,NaN,football,NaN,NaN,football,76,2018091001,3121,left,0.4
609726,2018-09-11T04:46:10.300,85.43,39.70,7.36,3.60,0.85,NaN,NaN,None,NaN,football,NaN,NaN,football,77,2018091001,3121,left,0.3
609727,2018-09-11T04:46:10.400,86.16,39.68,5.48,5.61,0.73,NaN,NaN,None,NaN,football,NaN,NaN,football,78,2018091001,3121,left,0.2
609728,2018-09-11T04:46:10.500,86.86,39.63,4.95,5.37,0.70,NaN,NaN,None,NaN,football,NaN,NaN,football,79,2018091001,3121,left,0.1
